In [1]:
import numpy as np
import json
import torch 
import math
import random

if torch.cuda.is_available() & False:
    print('Its Cudaaaaa time')
    torch.set_default_device('cuda')
else:
    torch.set_default_device('cpu')
    print("awww :( no cuda")


# before turning this on make sure all vectors are gpu tensors and all operations are GPU friendly
# also look into batching data for further optimizations.

awww :( no cuda


In [32]:
#HYPER PARAMATERS

K_PULL = 1
K_PUSH = 10
FORCE_GOAL = 0.0

In [3]:
with open('data\mpd.slice.0-999.json', 'r') as file:
    data = json.load(file)



<>:1: SyntaxWarning: invalid escape sequence '\m'
<>:1: SyntaxWarning: invalid escape sequence '\m'
C:\Users\benja\AppData\Local\Temp\ipykernel_4300\2319018493.py:1: SyntaxWarning: invalid escape sequence '\m'
  with open('data\mpd.slice.0-999.json', 'r') as file:


In [17]:
songsSet = set()

for playlist in data['playlists']:
    for track in playlist['tracks']:
        songsSet.add(track['track_name'])
songsList = list(songsSet)
songsList.sort()

strengthIndexes = {}
for idx, song in enumerate(songsList):
    strengthIndexes[song] = idx

songNum = len(songsList)

unMasked = torch.zeros(songNum,songNum)
mask = torch.triu(torch.ones([songNum,songNum], dtype=bool), diagonal=1)
connectionStrength = unMasked[mask]


for playlist in data['playlists']:
    for idx, track in enumerate(playlist['tracks']):
        for i in range(idx+1,len(playlist['tracks'])):

            idx1 = strengthIndexes[track['track_name']]
            idx2 = strengthIndexes[playlist['tracks'][i]['track_name']]

            a = min(idx1,idx2)
            b = max(idx1,idx2)
            #print(a,b)
            index = (2 * songNum - a - 1) * a // 2 + (b - a - 1)
            #print(index)

            connectionStrength[idx] += 1



In [18]:
print(connectionStrength)
print(len(connectionStrength))

tensor([66503., 65503., 64503.,  ...,     0.,     0.,     0.])
451456176


In [33]:
def pullForce(connectionVector: torch.tensor, connectionStrength : torch.tensor) -> torch.tensor:
    distances = torch.linalg.norm(connectionVector, dim=-1)
    #force = (1/2) * K_PULL * (distances ** 2)
    force = K_PULL * distances
    weightedForce = torch.mul(force, connectionStrength)  
    return weightedForce 

In [34]:
def pushForce(connectionVector : torch.tensor, connectionStrength : torch.tensor) -> torch.tensor:
    distances = torch.linalg.norm(connectionVector)
    #distances = torch.clamp(distances, min=1e-8)
    #force = -K_PUSH * torch.log(distances)
    force = -K_PUSH * distances
    #weightedForce = torch.mul(force, connectionStrength)  
    return force

In [21]:
def calculateForce(songCordinates : torch.tensor, connectionStrength : torch.tensor) -> torch.tensor: #output is pytorch tensor with single float in it
     connectionDistances = songCordinates[:, None, :] - songCordinates[None, :, :]
     uniqueConnectionDistances = connectionDistances[mask]
     forceTotal = torch.sum(pullForce(uniqueConnectionDistances,connectionStrength)) + torch.sum(pushForce(uniqueConnectionDistances,connectionStrength))
     return forceTotal

In [22]:
class CustomModel(torch.nn.Module):
    def __init__(self, songs : list, connectionStrength : torch.tensor):
        super(CustomModel, self).__init__()
        self.songCordinates = torch.nn.Parameter(torch.randn(len(songs),2,requires_grad=True)) #controls model dimension
        self.connectionStrength = connectionStrength
        self.songs = songs

    def forward(self):
        output = calculateForce(self.songCordinates, self.connectionStrength)
        return output

In [42]:


#MODEL PARAMATERS
EPOCHS = 50
LEARNING_RATE = 0.05
LOSS_FUNCTION = torch.nn.MSELoss()

model = CustomModel(songsList,connectionStrength) #fill in stuff
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

In [43]:
for epoch in range(EPOCHS):
    loss = LOSS_FUNCTION(model(),torch.tensor(FORCE_GOAL))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print(epoch)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [44]:
print(model.songCordinates)

Parameter containing:
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        ...,
        [nan, nan],
        [nan, nan],
        [nan, nan]], requires_grad=True)


In [41]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from sklearn.cluster import KMeans
import pandas as pd

def downsample_points(points, labels, max_points=10000):
    """
    Downsample points using K-means clustering if the dataset is too large.
    Returns both the downsampled points and the indices used for downsampling.
    """
    if len(points) <= max_points:
        return points, labels, np.arange(len(points))
    
    # Use K-means for intelligent downsampling
    kmeans = KMeans(n_clusters=max_points, random_state=42)
    kmeans.fit(points)
    
    # Find the closest points to each cluster center
    distances = np.linalg.norm(points[:, np.newaxis] - kmeans.cluster_centers_, axis=2)
    closest_point_indices = np.argmin(distances, axis=0)
    
    return points[closest_point_indices], np.array(labels)[closest_point_indices], closest_point_indices

def create_interactive_plot(points, labels, title="Labeled Points", downsample_threshold=10000):
    """
    Create an interactive Plotly scatter plot with optional downsampling
    """
    # Convert points and labels to pandas DataFrame for easier plotting
    if len(points) > downsample_threshold:
        print(f"Downsampling from {len(points)} to {downsample_threshold} points...")
        points_plot, labels_plot, _ = downsample_points(points, labels, downsample_threshold)
    else:
        points_plot, labels_plot = points, labels
    
    df = pd.DataFrame({
        'x': points_plot[:, 0],
        'y': points_plot[:, 1],
        'label': labels_plot
    })
    
    # Create the interactive plot
    fig = px.scatter(
        df,
        x='x',
        y='y',
        hover_data=['label'],
        title=title,
        labels={'x': 'X Coordinate', 'y': 'Y Coordinate'},
    )
    
    # Customize the layout
    fig.update_traces(
        marker=dict(size=6),
        hovertemplate="<b>Label:</b> %{customdata[0]}<br>" +
                     "<b>X:</b> %{x:.2f}<br>" +
                     "<b>Y:</b> %{y:.2f}<br>" +
                     "<extra></extra>"  # This removes the secondary box
    )
    
    fig.update_layout(
        template='plotly_white',
        hovermode='closest',
        width=900,
        height=700,
        showlegend=False,
        title={
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        }
    )
    
    # Add gridlines
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
    
    return fig

# Example usage with your data:
points = model.songCordinates.detach().numpy()
labels = [label.replace('$', '') for label in model.songs]

# Create and display the plot
fig = create_interactive_plot(points, labels, "Song Coordinates Visualization")
fig.show()

Downsampling from 30049 to 10000 points...


Apply markdown cells where needed